El siguiente kaggle trata de predecir las transacciones bancarias fraudaulentas con tarjetas de credito.
Los principales obstaculos tras un primer analisis de los datos son los siguientes:
* Solo un 3% de los datos son fraudulentos
* Desconocimiento de cada varible del dataset, ya que los datos son anonimizados
* Varibales con muchos valores nulos, al no saber el origen de la variable se hace dificil el tratamiento de los valores nulos
* Mas de 300 variables (Se propone una reduccion de varibles con PCAs)

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
transaction = pd.read_csv("train_transaction.csv")
identity = pd.read_csv("train_identity.csv")
transaction_test = pd.read_csv("test_transaction.csv")
identity_test = pd.read_csv("test_identity.csv")
#-------------------------------------TRANSACTION-------------------------------------#
transaction = transaction[transaction.TransactionAmt <= 800]

transaction.loc[transaction.card3.isin(transaction.card3.value_counts()[transaction.card3.value_counts() < 200].index), 'card3'] = "Others"
transaction.loc[transaction.card5.isin(transaction.card5.value_counts()[transaction.card5.value_counts() < 300].index), 'card5'] = "Others"
transaction.loc[transaction.addr1.isin(transaction.addr1.value_counts()[transaction.addr1.value_counts() <= 5000 ].index), 'addr1'] = "Others"
transaction.loc[transaction.addr2.isin(transaction.addr2.value_counts()[transaction.addr2.value_counts() <= 50 ].index), 'addr2'] = "Others"
transaction.loc[transaction['P_emaildomain'].isin(['gmail.com', 'gmail']),'P_emaildomain'] = 'Google'
transaction.loc[transaction['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk','yahoo.co.jp', 'yahoo.de', 'yahoo.fr','yahoo.es']), 'P_emaildomain'] = 'Yahoo Mail'
transaction.loc[transaction['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx','hotmail.es','hotmail.co.uk', 'hotmail.de',
                                         'outlook.es', 'live.com', 'live.fr','hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
transaction.loc[transaction.P_emaildomain.isin(transaction.P_emaildomain.value_counts()[transaction.P_emaildomain.value_counts() <= 500 ].index), 'P_emaildomain'] = "Others"
transaction.P_emaildomain.fillna("NoInf", inplace=True)
transaction.loc[transaction['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'
transaction.loc[transaction['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk','yahoo.co.jp', 'yahoo.de', 'yahoo.fr','yahoo.es']), 'R_emaildomain'] = 'Yahoo Mail'
transaction.loc[transaction['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx','hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr','hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
transaction.loc[transaction.R_emaildomain.isin(transaction.R_emaildomain.value_counts()[transaction.R_emaildomain.value_counts() <= 300 ].index), 'R_emaildomain'] = "Others"
transaction.R_emaildomain.fillna("NoInf", inplace=True)
transaction.loc[transaction.C1.isin(transaction.C1.value_counts()[transaction.C1.value_counts() <= 350 ].index), 'C1']= "Others"
transaction.loc[transaction.C2.isin(transaction.C2.value_counts()[transaction.C2.value_counts() <= 350 ].index), 'C2']= "Others"
transaction.loc[transaction.C3.isin(transaction.C3.value_counts()[transaction.C3.value_counts() <= 350 ].index), 'C3']= "Others"
transaction.loc[transaction.C4.isin(transaction.C4.value_counts()[transaction.C4.value_counts() <= 200 ].index), 'C4']= "Others"
transaction.loc[transaction.C5.isin(transaction.C5.value_counts()[transaction.C5.value_counts() <= 300 ].index), 'C5']= "Others"
transaction.loc[transaction.C6[transaction.C6 > 58 ].index, 'C6'] ="Others"
transaction.loc[transaction.C7[transaction.C7 > 20 ].index, 'C7'] ="Others"
transaction.loc[transaction.C8.isin(transaction.C8.value_counts()[transaction.C8.value_counts() <= 60 ].index), 'C8']= "Others"
transaction.loc[transaction.C9.isin(transaction.C9.value_counts()[transaction.C9.value_counts() <= 400 ].index), 'C9']= "Others"
transaction.loc[transaction.C10.isin(transaction.C10.value_counts()[transaction.C10.value_counts() <= 100 ].index), 'C10']= "Others"
transaction.loc[transaction.C11.isin(transaction.C11.value_counts()[transaction.C11.value_counts() <= 430 ].index), 'C11']= "Others"
transaction.loc[transaction.C12.isin(transaction.C12.value_counts()[transaction.C12.value_counts() <= 50 ].index), 'C12']= "Others"
transaction.loc[transaction.C13.isin(transaction.C13.value_counts()[transaction.C13.value_counts() <= 600 ].index), 'C13']= "Others"
transaction.loc[transaction.C14.isin(transaction.C14.value_counts()[transaction.C14.value_counts() <= 400 ].index), 'C14']= "Others"
#----------------------------TRANSACTION TEZT--------------------------------------
transaction_test.loc[transaction_test.card3.isin(transaction_test.card3.value_counts()[transaction_test.card3.value_counts() < 200].index), 'card3'] = "Others"
transaction_test.loc[transaction_test.card5.isin(transaction_test.card5.value_counts()[transaction_test.card5.value_counts() < 300].index), 'card5'] = "Others"
transaction_test.loc[transaction_test.addr1.isin(transaction_test.addr1.value_counts()[transaction_test.addr1.value_counts() <= 5000 ].index), 'addr1'] = "Others"
transaction_test.loc[transaction_test.addr2.isin(transaction_test.addr2.value_counts()[transaction_test.addr2.value_counts() <= 50 ].index), 'addr2'] = "Others"
transaction_test.loc[transaction_test['P_emaildomain'].isin(['gmail.com', 'gmail']),'P_emaildomain'] = 'Google'
transaction_test.loc[transaction_test['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk','yahoo.co.jp', 'yahoo.de', 'yahoo.fr','yahoo.es']), 'P_emaildomain'] = 'Yahoo Mail'
transaction_test.loc[transaction_test['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx','hotmail.es','hotmail.co.uk', 'hotmail.de',
                                         'outlook.es', 'live.com', 'live.fr','hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
transaction_test.loc[transaction_test.P_emaildomain.isin(transaction_test.P_emaildomain.value_counts()[transaction_test.P_emaildomain.value_counts() <= 500 ].index), 'P_emaildomain'] = "Others"
transaction_test.P_emaildomain.fillna("NoInf", inplace=True)
transaction_test.loc[transaction_test['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'
transaction_test.loc[transaction_test['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk','yahoo.co.jp', 'yahoo.de', 'yahoo.fr','yahoo.es']), 'R_emaildomain'] = 'Yahoo Mail'
transaction_test.loc[transaction_test['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx','hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr','hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
transaction_test.loc[transaction_test.R_emaildomain.isin(transaction_test.R_emaildomain.value_counts()[transaction_test.R_emaildomain.value_counts() <= 300 ].index), 'R_emaildomain'] = "Others"
transaction_test.R_emaildomain.fillna("NoInf", inplace=True)
transaction_test.loc[transaction_test.C1.isin(transaction_test.C1.value_counts()[transaction_test.C1.value_counts() <= 350 ].index), 'C1']= "Others"
transaction_test.loc[transaction_test.C2.isin(transaction_test.C2.value_counts()[transaction_test.C2.value_counts() <= 350 ].index), 'C2']= "Others"
transaction_test.loc[transaction_test.C3.isin(transaction_test.C3.value_counts()[transaction_test.C3.value_counts() <= 350 ].index), 'C3']= "Others"
transaction_test.loc[transaction_test.C4.isin(transaction_test.C4.value_counts()[transaction_test.C4.value_counts() <= 200 ].index), 'C4']= "Others"
transaction_test.loc[transaction_test.C5.isin(transaction_test.C5.value_counts()[transaction_test.C5.value_counts() <= 300 ].index), 'C5']= "Others"
transaction_test.loc[transaction_test.C6[transaction_test.C6 > 58 ].index, 'C6'] ="Others"
transaction_test.loc[transaction_test.C7[transaction_test.C7 > 20 ].index, 'C7'] ="Others"
transaction_test.loc[transaction_test.C8.isin(transaction_test.C8.value_counts()[transaction_test.C8.value_counts() <= 60 ].index), 'C8']= "Others"
transaction_test.loc[transaction_test.C9.isin(transaction_test.C9.value_counts()[transaction_test.C9.value_counts() <= 400 ].index), 'C9']= "Others"
transaction_test.loc[transaction_test.C10.isin(transaction_test.C10.value_counts()[transaction_test.C10.value_counts() <= 100 ].index), 'C10']= "Others"
transaction_test.loc[transaction_test.C11.isin(transaction_test.C11.value_counts()[transaction_test.C11.value_counts() <= 430 ].index), 'C11']= "Others"
transaction_test.loc[transaction_test.C12.isin(transaction_test.C12.value_counts()[transaction_test.C12.value_counts() <= 50 ].index), 'C12']= "Others"
transaction_test.loc[transaction_test.C13.isin(transaction_test.C13.value_counts()[transaction_test.C13.value_counts() <= 600 ].index), 'C13']= "Others"
transaction_test.loc[transaction_test.C14.isin(transaction_test.C14.value_counts()[transaction_test.C14.value_counts() <= 400 ].index), 'C14']= "Others"

#--------------------------IDENTITY---------------------------------------------#
identity['id_30'] = np.where((identity.id_31 == 'Generic/Android')
                            |(identity.id_31 == 'Generic/Android 7.0')
                            |(identity.id_31 == 'BLU/Dash')
                            |(identity.id_31 == 'Lanix/Ilium')
                            |(identity.id_31 == 'M4Tel/M4')
                            |(identity.id_31 == 'ZTE/Blade')
                            |(identity.id_31 == 'LG/K-200')
                            |(identity.id_31 == 'Samsung/SCH')
                            |(identity.id_31 == 'Samsung/SM-G531H')
                            |(identity.id_31 == 'Samsung/SM-G532M')
                            |(identity.id_31 == 'android')
                            |(identity.id_31 == 'android browser 4.0')
                            |(identity.id_31 == 'chrome 43.0 for android')
                            |(identity.id_31 == 'chrome 46.0 for android')
                            |(identity.id_31 == 'chrome 49.0 for android')
                            |(identity.id_31 == 'chrome 50.0 for android')
                            |(identity.id_31 == 'chrome 51.0 for android')
                            |(identity.id_31 == 'chrome 52.0 for android')
                            |(identity.id_31 == 'chrome 53.0 for android')
                            |(identity.id_31 == 'chrome 54.0 for android')
                            |(identity.id_31 == 'chrome 55.0 for android')
                            |(identity.id_31 == 'chrome 56.0 for android')
                            |(identity.id_31 == 'chrome 57.0 for android')
                            |(identity.id_31 == 'chrome 58.0 for android')
                            |(identity.id_31 == 'chrome 59.0 for android')
                            |(identity.id_31 == 'chrome 60.0 for android')
                            |(identity.id_31 == 'chrome 61.0 for android')
                            |(identity.id_31 == 'chrome 62.0 for android')
                            |(identity.id_31 == 'chrome 63.0 for android')
                            |(identity.id_31 == 'chrome 64.0 for android')
                            |(identity.id_31 == 'chrome 65.0 for android')
                            |(identity.id_31 == 'chrome 66.0 for android')
                            |(identity.id_31 == 'chrome 67.0 for android')
                            |(identity.id_31 == 'chrome generic for android')
                            |(identity.id_31 == 'line')
                            |(identity.id_31 == 'other')
                            |(identity.id_31 == 'samsung')
                            |(identity.id_31 == 'samsung browser 3.3')
                            |(identity.id_31 == 'samsung browser 4.0')
                            |(identity.id_31 == 'samsung browser 4.2')
                            |(identity.id_31 == 'samsung browser 5.2')
                            |(identity.id_31 == 'samsung browser 5.4')
                            |(identity.id_31 == 'samsung browser 6.2')
                            |(identity.id_31 == 'samsung browser 6.4')
                            |(identity.id_31 == 'samsung browser 7.0')
                            |(identity.id_31 == 'samsung browser generic'),'Android',identity.id_30)
identity['id_30'] = np.where((identity.id_31 == 'chrome 62.0 for ios')
                            |(identity.id_31 == 'chrome 63.0 for ios')
                            |(identity.id_31 == 'chrome 64.0 for ios')
                            |(identity.id_31 == 'chrome 65.0 for ios')
                            |(identity.id_31 == 'chrome 66.0 for ios')
                            |(identity.id_31 == 'google search application 48.0')
                            |(identity.id_31 == 'google search application 49.0')
                            |(identity.id_31 == 'mobile')
                            |(identity.id_31 == 'mobile safari 10.0')
                            |(identity.id_31 == 'mobile safari 11.0')
                            |(identity.id_31 == 'mobile safari 8.0')
                            |(identity.id_31 == 'mobile safari 9.0')
                            |(identity.id_31 == 'mobile safari generic')
                            |(identity.id_31 == 'mobile safari uiwebview'),'iOS',identity.id_30)
identity['id_30'] = np.where((identity.id_31 == 'Microsoft/Windows')
                            |(identity.id_31 == 'comodo')
                            |(identity.id_31 == 'edge')
                            |(identity.id_31 == 'edge 13.0')
                            |(identity.id_31 == 'edge 14.0')
                            |(identity.id_31 == 'edge 15.0')
                            |(identity.id_31 == 'edge 16.0')
                            |(identity.id_31 == 'edge 17.0')
                            |(identity.id_31 == 'ie')
                            |(identity.id_31 == 'ie 11.0 for desktop')
                            |(identity.id_31 == 'ie 11.0 for tablet')
                            |(identity.id_31 == 'opera 52.0')
                            |(identity.id_31 == 'opera 53.0')
                            |(identity.id_31 == 'opera generic'),'Windows',identity.id_30)
identity['id_30'] = np.where((identity.id_31 == 'safari')
                            |(identity.id_31 == 'safari 10.0')
                            |(identity.id_31 == 'safari 11.0')
                            |(identity.id_31 == 'safari 9.0')
                            |(identity.id_31 == 'safari generic'),'Mac',identity.id_30)
identity['id_30'] = np.where((identity.id_31 == 'Nokia/Lumia')
                            |(identity.id_31 == 'other'),'other',identity.id_30)
identity['id_30'] = np.where((identity.id_31 == 'chromium'),'Linux',identity.id_30)
identity['id_30'] = np.where((identity.DeviceInfo == 'iOS Device'),'iOS',identity.id_30)
identity['id_30'] = np.where((identity.DeviceInfo == 'Windows'),'Windows',identity.id_30)
identity['id_30'] = np.where((identity.DeviceInfo == 'MacOS'),'Mac',identity.id_30)
identity['id_30'] = np.where((identity.id_30.isnull())&(identity.DeviceInfo.notnull()&(identity.DeviceType == "mobile")),'Android',identity.id_30)
var = identity["DeviceInfo"].str.split(" ", n = 1, expand = True)
identity['var'] = var[0]
identity['id_30'] = np.where((identity.id_30.isnull())&(identity.DeviceInfo.notnull())&(identity.DeviceType == "desktop")
                                                       |(identity['var'] == 'Moto')
                                                       |(identity['var'] == 'LG-D320')
                                                       |(identity['var'] == 'HTC')
                                                       |(identity['var'] == 'SAMSUNG')
                                                       |(identity['var'] == 'SAMSUNG-SM-G530AZ')
                                                       |(identity['var'] == 'Android')
                                                       |(identity['var'] == 'moto')
                                                       |(identity['var'] == 'MotoG3')
                                                       |(identity['var'] == 'Nexus')
                                                       |(identity['var'] == 'HUAWEI'),'Android',identity.id_30)
#--------------------------------------IDENTITY TEST ---------------------------------

identity_test['id_30'] = np.where((identity_test.id_31 == 'Generic/Android')
                            |(identity_test.id_31 == 'Generic/Android 7.0')
                            |(identity_test.id_31 == 'BLU/Dash')
                            |(identity_test.id_31 == 'Lanix/Ilium')
                            |(identity_test.id_31 == 'M4Tel/M4')
                            |(identity_test.id_31 == 'ZTE/Blade')
                            |(identity_test.id_31 == 'LG/K-200')
                            |(identity_test.id_31 == 'Samsung/SCH')
                            |(identity_test.id_31 == 'Samsung/SM-G531H')
                            |(identity_test.id_31 == 'Samsung/SM-G532M')
                            |(identity_test.id_31 == 'android')
                            |(identity_test.id_31 == 'android browser 4.0')
                            |(identity_test.id_31 == 'chrome 43.0 for android')
                            |(identity_test.id_31 == 'chrome 46.0 for android')
                            |(identity_test.id_31 == 'chrome 49.0 for android')
                            |(identity_test.id_31 == 'chrome 50.0 for android')
                            |(identity_test.id_31 == 'chrome 51.0 for android')
                            |(identity_test.id_31 == 'chrome 52.0 for android')
                            |(identity_test.id_31 == 'chrome 53.0 for android')
                            |(identity_test.id_31 == 'chrome 54.0 for android')
                            |(identity_test.id_31 == 'chrome 55.0 for android')
                            |(identity_test.id_31 == 'chrome 56.0 for android')
                            |(identity_test.id_31 == 'chrome 57.0 for android')
                            |(identity_test.id_31 == 'chrome 58.0 for android')
                            |(identity_test.id_31 == 'chrome 59.0 for android')
                            |(identity_test.id_31 == 'chrome 60.0 for android')
                            |(identity_test.id_31 == 'chrome 61.0 for android')
                            |(identity_test.id_31 == 'chrome 62.0 for android')
                            |(identity_test.id_31 == 'chrome 63.0 for android')
                            |(identity_test.id_31 == 'chrome 64.0 for android')
                            |(identity_test.id_31 == 'chrome 65.0 for android')
                            |(identity_test.id_31 == 'chrome 66.0 for android')
                            |(identity_test.id_31 == 'chrome 67.0 for android')
                            |(identity_test.id_31 == 'chrome generic for android')
                            |(identity_test.id_31 == 'line')
                            |(identity_test.id_31 == 'other')
                            |(identity_test.id_31 == 'samsung')
                            |(identity_test.id_31 == 'samsung browser 3.3')
                            |(identity_test.id_31 == 'samsung browser 4.0')
                            |(identity_test.id_31 == 'samsung browser 4.2')
                            |(identity_test.id_31 == 'samsung browser 5.2')
                            |(identity_test.id_31 == 'samsung browser 5.4')
                            |(identity_test.id_31 == 'samsung browser 6.2')
                            |(identity_test.id_31 == 'samsung browser 6.4')
                            |(identity_test.id_31 == 'samsung browser 7.0')
                            |(identity_test.id_31 == 'samsung browser generic'),'Android',identity_test.id_30)
identity_test['id_30'] = np.where((identity_test.id_31 == 'chrome 62.0 for ios')
                            |(identity_test.id_31 == 'chrome 63.0 for ios')
                            |(identity_test.id_31 == 'chrome 64.0 for ios')
                            |(identity_test.id_31 == 'chrome 65.0 for ios')
                            |(identity_test.id_31 == 'chrome 66.0 for ios')
                            |(identity_test.id_31 == 'google search application 48.0')
                            |(identity_test.id_31 == 'google search application 49.0')
                            |(identity_test.id_31 == 'mobile')
                            |(identity_test.id_31 == 'mobile safari 10.0')
                            |(identity_test.id_31 == 'mobile safari 11.0')
                            |(identity_test.id_31 == 'mobile safari 8.0')
                            |(identity_test.id_31 == 'mobile safari 9.0')
                            |(identity_test.id_31 == 'mobile safari generic')
                            |(identity_test.id_31 == 'mobile safari uiwebview'),'iOS',identity_test.id_30)
identity_test['id_30'] = np.where((identity_test.id_31 == 'Microsoft/Windows')
                            |(identity_test.id_31 == 'comodo')
                            |(identity_test.id_31 == 'edge')
                            |(identity_test.id_31 == 'edge 13.0')
                            |(identity_test.id_31 == 'edge 14.0')
                            |(identity_test.id_31 == 'edge 15.0')
                            |(identity_test.id_31 == 'edge 16.0')
                            |(identity_test.id_31 == 'edge 17.0')
                            |(identity_test.id_31 == 'ie')
                            |(identity_test.id_31 == 'ie 11.0 for desktop')
                            |(identity_test.id_31 == 'ie 11.0 for tablet')
                            |(identity_test.id_31 == 'opera 52.0')
                            |(identity_test.id_31 == 'opera 53.0')
                            |(identity_test.id_31 == 'opera generic'),'Windows',identity_test.id_30)
identity_test['id_30'] = np.where((identity_test.id_31 == 'safari')
                            |(identity_test.id_31 == 'safari 10.0')
                            |(identity_test.id_31 == 'safari 11.0')
                            |(identity_test.id_31 == 'safari 9.0')
                            |(identity_test.id_31 == 'safari generic'),'Mac',identity_test.id_30)
identity_test['id_30'] = np.where((identity_test.id_31 == 'Nokia/Lumia')
                            |(identity_test.id_31 == 'other'),'other',identity_test.id_30)
identity_test['id_30'] = np.where((identity_test.id_31 == 'chromium'),'Linux',identity_test.id_30)
identity_test['id_30'] = np.where((identity_test.DeviceInfo == 'iOS Device'),'iOS',identity_test.id_30)
identity_test['id_30'] = np.where((identity_test.DeviceInfo == 'Windows'),'Windows',identity_test.id_30)
identity_test['id_30'] = np.where((identity_test.DeviceInfo == 'MacOS'),'Mac',identity_test.id_30)
identity_test['id_30'] = np.where((identity_test.id_30.isnull())&(identity_test.DeviceInfo.notnull()&(identity_test.DeviceType == "mobile")),'Android',identity_test.id_30)
var = identity_test["DeviceInfo"].str.split(" ", n = 1, expand = True)
identity_test['var'] = var[0]
identity_test['id_30'] = np.where((identity_test.id_30.isnull())&(identity_test.DeviceInfo.notnull())&(identity_test.DeviceType == "desktop")
                                                       |(identity_test['var'] == 'Moto')
                                                       |(identity_test['var'] == 'LG-D320')
                                                       |(identity_test['var'] == 'HTC')
                                                       |(identity_test['var'] == 'SAMSUNG')
                                                       |(identity_test['var'] == 'SAMSUNG-SM-G530AZ')
                                                       |(identity_test['var'] == 'Android')
                                                       |(identity_test['var'] == 'moto')
                                                       |(identity_test['var'] == 'MotoG3')
                                                       |(identity_test['var'] == 'Nexus')
                                                       |(identity_test['var'] == 'HUAWEI'),'Android',identity_test.id_30)
col_cat_identity =['id_30','id_34','id_35','id_36','id_37','id_38','DeviceType']
col_target =['isFraud']
col_id = ['TransactionID']
col_num_identity = ['id_01','id_02','id_05','id_06','id_07','id_08','id_09','id_12','id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29']
identity = identity[col_cat_identity+col_id+col_num_identity]
identity_test = identity_test[col_cat_identity+col_id+col_num_identity]

In [2]:
Full_transaction = transaction.merge(identity, how='left', left_index=True, right_index=True, on='TransactionID')
Full_transaction_test = transaction_test.merge(identity, how='left', left_index=True, right_index=True, on='TransactionID')
Full_transaction['TransactionAmt'] = np.log(Full_transaction['TransactionAmt'])
Full_transaction_test['TransactionAmt'] = np.log(Full_transaction['TransactionAmt'])
Id_col     = ['TransactionID']
target_col = ["isFraud"]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


In [3]:
Full_transaction = Full_transaction.reset_index().drop(['index'], axis=1)
Full_transaction_test = Full_transaction_test.reset_index().drop(['index'], axis=1)

In [4]:
from sklearn import preprocessing
for f in Full_transaction.drop('isFraud', axis=1).columns:
    if Full_transaction[f].dtype=='object' or Full_transaction_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(Full_transaction[f].values) + list(Full_transaction_test[f].values))
        Full_transaction[f] = lbl.transform(list(Full_transaction[f].values))
        Full_transaction_test[f] = lbl.transform(list(Full_transaction_test[f].values))   

In [5]:
Full_transaction['Trans_min_mean'] = Full_transaction['TransactionAmt'] - Full_transaction['TransactionAmt'].mean()
Full_transaction['Trans_min_std'] = Full_transaction['Trans_min_mean'] / Full_transaction['TransactionAmt'].std()
Full_transaction_test['Trans_min_mean'] = Full_transaction_test['TransactionAmt'] - Full_transaction_test['TransactionAmt'].mean()
Full_transaction_test['Trans_min_std'] = Full_transaction_test['Trans_min_mean'] / Full_transaction_test['TransactionAmt'].std()
Full_transaction['TransactionAmt_to_mean_card1'] = Full_transaction['TransactionAmt'] / Full_transaction.groupby(['card1'])['TransactionAmt'].transform('mean')
Full_transaction['TransactionAmt_to_mean_card2'] = Full_transaction['TransactionAmt'] / Full_transaction.groupby(['card2'])['TransactionAmt'].transform('mean')
Full_transaction['TransactionAmt_to_mean_card4'] = Full_transaction['TransactionAmt'] / Full_transaction.groupby(['card4'])['TransactionAmt'].transform('mean')
Full_transaction['TransactionAmt_to_std_card1'] = Full_transaction['TransactionAmt'] / Full_transaction.groupby(['card1'])['TransactionAmt'].transform('std')
Full_transaction['TransactionAmt_to_std_card2'] = Full_transaction['TransactionAmt'] / Full_transaction.groupby(['card2'])['TransactionAmt'].transform('std')
Full_transaction['TransactionAmt_to_std_card4'] = Full_transaction['TransactionAmt'] / Full_transaction.groupby(['card4'])['TransactionAmt'].transform('std')

Full_transaction_test['TransactionAmt_to_mean_card1'] = Full_transaction_test['TransactionAmt'] / Full_transaction_test.groupby(['card1'])['TransactionAmt'].transform('mean')
Full_transaction_test['TransactionAmt_to_mean_card2'] = Full_transaction_test['TransactionAmt'] / Full_transaction_test.groupby(['card2'])['TransactionAmt'].transform('mean')
Full_transaction_test['TransactionAmt_to_mean_card4'] = Full_transaction_test['TransactionAmt'] / Full_transaction_test.groupby(['card4'])['TransactionAmt'].transform('mean')
Full_transaction_test['TransactionAmt_to_std_card1'] = Full_transaction_test['TransactionAmt'] / Full_transaction_test.groupby(['card1'])['TransactionAmt'].transform('std')
Full_transaction_test['TransactionAmt_to_std_card2'] = Full_transaction_test['TransactionAmt'] / Full_transaction_test.groupby(['card2'])['TransactionAmt'].transform('std')
Full_transaction_test['TransactionAmt_to_std_card4'] = Full_transaction_test['TransactionAmt'] / Full_transaction_test.groupby(['card4'])['TransactionAmt'].transform('std')

Full_transaction['id_02_to_mean_card1'] = Full_transaction['id_02'] / Full_transaction.groupby(['card1'])['id_02'].transform('mean')
Full_transaction['id_02_to_mean_card2'] = Full_transaction['id_02'] / Full_transaction.groupby(['card2'])['id_02'].transform('mean')
Full_transaction['id_02_to_mean_card4'] = Full_transaction['id_02'] / Full_transaction.groupby(['card4'])['id_02'].transform('mean')
Full_transaction['id_02_to_std_card1'] = Full_transaction['id_02'] / Full_transaction.groupby(['card1'])['id_02'].transform('std')
Full_transaction['id_02_to_std_card2'] = Full_transaction['id_02'] / Full_transaction.groupby(['card2'])['id_02'].transform('std')
Full_transaction['id_02_to_std_card4'] = Full_transaction['id_02'] / Full_transaction.groupby(['card4'])['id_02'].transform('std')

Full_transaction_test['id_02_to_mean_card1'] = Full_transaction_test['id_02'] / Full_transaction_test.groupby(['card1'])['id_02'].transform('mean')
Full_transaction_test['id_02_to_mean_card2'] = Full_transaction_test['id_02'] / Full_transaction_test.groupby(['card2'])['id_02'].transform('mean')
Full_transaction_test['id_02_to_mean_card4'] = Full_transaction_test['id_02'] / Full_transaction_test.groupby(['card4'])['id_02'].transform('mean')
Full_transaction_test['id_02_to_std_card1'] = Full_transaction_test['id_02'] / Full_transaction_test.groupby(['card1'])['id_02'].transform('std')
Full_transaction_test['id_02_to_std_card2'] = Full_transaction_test['id_02'] / Full_transaction_test.groupby(['card2'])['id_02'].transform('std')
Full_transaction_test['id_02_to_std_card4'] = Full_transaction_test['id_02'] / Full_transaction_test.groupby(['card4'])['id_02'].transform('std')

In [6]:
Full_transaction_test['isFraud'] = 'test'
df = pd.concat([Full_transaction, Full_transaction_test], axis=0, sort=False )
df = df.reset_index()
df = df.drop('index', axis=1)

In [7]:
mas_v = df.columns[55:394]

In [8]:
#Funcion de PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import minmax_scale

def PCA_change(df, cols, n_components, prefix='PCA_', rand_seed=4):
    pca = PCA(n_components=n_components, random_state=rand_seed)

    principalComponents = pca.fit_transform(df[cols])

    principalDf = pd.DataFrame(principalComponents)

    df.drop(cols, axis=1, inplace=True)

    principalDf.rename(columns=lambda x: str(prefix)+str(x), inplace=True)

    df = pd.concat([df, principalDf], axis=1)
    
    return df

In [9]:
for col in mas_v:
    df[col] = df[col].fillna((df[col].min() - 2))
    df[col] = (minmax_scale(df[col], feature_range=(0,1)))

In [10]:
#Comprobar con cuantas columnas nos es util hacer PCA
pca = PCA(whiten=True)
pca.fit(df[mas_v])
variance = pd.DataFrame(pca.explained_variance_ratio_)
np.cumsum(pca.explained_variance_ratio_)

array([0.49533697, 0.72848424, 0.8251377 , 0.871538  , 0.89371866,
       0.91039998, 0.92594241, 0.93708206, 0.94521087, 0.95229277,
       0.95878277, 0.96211267, 0.96496257, 0.96723919, 0.96901706,
       0.97064072, 0.97221863, 0.97364838, 0.9750345 , 0.97622118,
       0.9773029 , 0.97836911, 0.9793213 , 0.9801852 , 0.9809906 ,
       0.98176108, 0.98246593, 0.98313537, 0.98375764, 0.98436524,
       0.98493366, 0.98548322, 0.98598336, 0.98646012, 0.98692293,
       0.98736293, 0.98779366, 0.98821854, 0.98859846, 0.98896667,
       0.98931935, 0.98965567, 0.98998653, 0.99029648, 0.99059556,
       0.9908751 , 0.99115095, 0.99141398, 0.99167299, 0.99191228,
       0.99214603, 0.992377  , 0.99259268, 0.99280258, 0.99299819,
       0.99318517, 0.99337012, 0.99354573, 0.99371511, 0.99388095,
       0.99404248, 0.99419691, 0.99434942, 0.99449235, 0.99463178,
       0.99476595, 0.99489729, 0.99502352, 0.99514665, 0.99526635,
       0.99538463, 0.9954985 , 0.99561015, 0.99572032, 0.99582

In [11]:
df = PCA_change(df, mas_v, prefix='PCA_V_', n_components=50)

In [12]:
df_train, df_test = df[df['isFraud'] != 'test'], df[df['isFraud'] == 'test'].drop('isFraud', axis=1)


In [13]:
df_test.head()
X_train = df_train.drop('isFraud', axis=1)
y_train = df_train['isFraud']
y_train = y_train.astype('int')




In [14]:
from sklearn.model_selection import KFold,TimeSeriesSplit
import gc
from sklearn.metrics import roc_auc_score
from xgboost import plot_importance
from sklearn.metrics import make_scorer
from hyperopt import fmin, tpe, hp, Trials, space_eval
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import time
def objective(params):
    time1 = time.time()
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'subsample': "{:.2f}".format(params['subsample']),
        'reg_alpha': "{:.3f}".format(params['reg_alpha']),
        'reg_lambda': "{:.3f}".format(params['reg_lambda']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'num_leaves': '{:.3f}'.format(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'min_child_samples': '{:.3f}'.format(params['min_child_samples']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction'])
    }

    print("\n############## New Run ################")
    print(f"params = {params}")
    FOLDS = 7
    count=1
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

    tss = TimeSeriesSplit(n_splits=FOLDS)
    score_mean = 0
    for tr_idx, val_idx in tss.split(X_train, y_train):
        clf = xgb.XGBClassifier(
            n_estimators=600, random_state=4, verbose=True, 
            tree_method='hist', 
            **params
        )

        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        
        clf.fit(X_tr, y_tr)
        #y_pred_train = clf.predict_proba(X_vl)[:,1]
        #print(y_pred_train)
        score = make_scorer(roc_auc_score, needs_proba=True)(clf, X_vl, y_vl)
        # plt.show()
        score_mean += score
        print(f'{count} CV - score: {round(score, 4)}')
        count += 1
    time2 = time.time() - time1
    print(f"Total Time Run: {round(time2 / 60,2)}")
    gc.collect()
    print(f'Mean ROC_AUC: {score_mean / FOLDS}')
    del X_tr, X_vl, y_tr, y_vl, clf, score
    return -(score_mean / FOLDS)


space = {
    # The maximum depth of a tree, same as GBM.
    # Used to control over-fitting as higher depth will allow model 
    # to learn relations very specific to a particular sample.
    # Should be tuned using CV.
    # Typical values: 3-10
    'max_depth': hp.quniform('max_depth', 7, 23, 1),
    
    # reg_alpha: L1 regularization term. L1 regularization encourages sparsity 
    # (meaning pulling weights to 0). It can be more useful when the objective
    # is logistic regression since you might need help with feature selection.
    'reg_alpha':  hp.uniform('reg_alpha', 0.01, 0.4),
    
    # reg_lambda: L2 regularization term. L2 encourages smaller weights, this
    # approach can be more useful in tree-models where zeroing 
    # features might not make much sense.
    'reg_lambda': hp.uniform('reg_lambda', 0.01, .4),
    
    # eta: Analogous to learning rate in GBM
    # Makes the model more robust by shrinking the weights on each step
    # Typical final values to be used: 0.01-0.2
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    
    # colsample_bytree: Similar to max_features in GBM. Denotes the 
    # fraction of columns to be randomly samples for each tree.
    # Typical values: 0.5-1
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, .9),
    
    # A node is split only when the resulting split gives a positive
    # reduction in the loss function. Gamma specifies the 
    # minimum loss reduction required to make a split.
    # Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
    'gamma': hp.uniform('gamma', 0.01, .7),
    
    # more increases accuracy, but may lead to overfitting.
    # num_leaves: the number of leaf nodes to use. Having a large number 
    # of leaves will improve accuracy, but will also lead to overfitting.
    'num_leaves': hp.choice('num_leaves', list(range(20, 250, 10))),
    
    # specifies the minimum samples per leaf node.
    # the minimum number of samples (data) to group into a leaf. 
    # The parameter can greatly assist with overfitting: larger sample
    # sizes per leaf will reduce overfitting (but may lead to under-fitting).
    'min_child_samples': hp.choice('min_child_samples', list(range(100, 250, 10))),
    
    # subsample: represents a fraction of the rows (observations) to be 
    # considered when building each subtree. Tianqi Chen and Carlos Guestrin
    # in their paper A Scalable Tree Boosting System recommend 
    'subsample': hp.choice('subsample', [0.2, 0.4, 0.5, 0.6, 0.7, .8, .9]),
    
    # randomly select a fraction of the features.
    # feature_fraction: controls the subsampling of features used
    # for training (as opposed to subsampling the actual training data in 
    # the case of bagging). Smaller fractions reduce overfitting.
    'feature_fraction': hp.uniform('feature_fraction', 0.4, .8),
    
    # randomly bag or subsample training data.
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, .9)
    
    # bagging_fraction and bagging_freq: enables bagging (subsampling) 
    # of the training data. Both values need to be set for bagging to be used.
    # The frequency controls how often (iteration) bagging is used. Smaller
    # fractions and frequencies reduce overfitting.
}
# Set algoritm parameters
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=60)

# Print best parameters
best_params = space_eval(space, best)

                                                    
############## New Run ################
params = {'max_depth': 23, 'gamma': '0.482', 'subsample': '0.20', 'reg_alpha': '0.025', 'reg_lambda': '0.192', 'learning_rate': '0.027', 'num_leaves': '100.000', 'colsample_bytree': '0.432', 'min_child_samples': '110.000', 'feature_fraction': '0.494', 'bagging_fraction': '0.592'}
1 CV - score: 0.8852                                
2 CV - score: 0.8886                                
3 CV - score: 0.9131                                
4 CV - score: 0.8791                                
5 CV - score: 0.9152                                
6 CV - score: 0.9073                                
7 CV - score: 0.907                                 
Total Time Run: 60.26                                 
Mean ROC_AUC: 0.8993570954442126                      
                                                                                     
############## New Run ################
params = {'max_dept

1 CV - score: 0.8891                                                                 
2 CV - score: 0.8882                                                                 
3 CV - score: 0.9133                                                                 
4 CV - score: 0.8791                                                                 
5 CV - score: 0.9124                                                                 
6 CV - score: 0.9107                                                                 
7 CV - score: 0.908                                                                  
Total Time Run: 57.51                                                                
Mean ROC_AUC: 0.9001132804482023                                                     
                                                                                     
############## New Run ################
params = {'max_depth': 8, 'gamma': '0.400', 'subsample': '0.40', 'reg_alpha': '0.287', 'reg_lambda':

1 CV - score: 0.8659                                                                   
2 CV - score: 0.8732                                                                   
3 CV - score: 0.894                                                                    
4 CV - score: 0.8532                                                                   
5 CV - score: 0.8978                                                                   
6 CV - score: 0.8871                                                                   
7 CV - score: 0.8894                                                                   
Total Time Run: 47.57                                                                  
Mean ROC_AUC: 0.8800742720403097                                                       
                                                                                       
############## New Run ################
params = {'max_depth': 17, 'gamma': '0.158', 'subsample': '0.60', 'reg_alpha': '

Mean ROC_AUC: 0.9147590740181704                                                       
                                                                                       
############## New Run ################
params = {'max_depth': 15, 'gamma': '0.029', 'subsample': '0.80', 'reg_alpha': '0.195', 'reg_lambda': '0.258', 'learning_rate': '0.028', 'num_leaves': '20.000', 'colsample_bytree': '0.821', 'min_child_samples': '160.000', 'feature_fraction': '0.681', 'bagging_fraction': '0.476'}
1 CV - score: 0.8999                                                                   
2 CV - score: 0.9059                                                                   
3 CV - score: 0.9257                                                                   
4 CV - score: 0.8988                                                                   
5 CV - score: 0.928                                                                    
6 CV - score: 0.9239                                            

5 CV - score: 0.9205                                                                  
6 CV - score: 0.9192                                                                  
7 CV - score: 0.9111                                                                  
Total Time Run: 64.48                                                                 
Mean ROC_AUC: 0.9068321945953175                                                      
                                                                                      
############## New Run ################
params = {'max_depth': 20, 'gamma': '0.606', 'subsample': '0.50', 'reg_alpha': '0.230', 'reg_lambda': '0.238', 'learning_rate': '0.066', 'num_leaves': '50.000', 'colsample_bytree': '0.778', 'min_child_samples': '110.000', 'feature_fraction': '0.777', 'bagging_fraction': '0.664'}
1 CV - score: 0.8907                                                                  
2 CV - score: 0.8947                                                   

1 CV - score: 0.8946                                                                 
2 CV - score: 0.8968                                                                 
3 CV - score: 0.9213                                                                 
4 CV - score: 0.89                                                                   
5 CV - score: 0.9236                                                                 
6 CV - score: 0.9146                                                                 
7 CV - score: 0.917                                                                  
Total Time Run: 70.96                                                                
Mean ROC_AUC: 0.9082647595699884                                                     
                                                                                     
############## New Run ################
params = {'max_depth': 10, 'gamma': '0.276', 'subsample': '0.60', 'reg_alpha': '0.032', 'reg_lambda'

In [15]:
print("BEST PARAMS: ", best_params)



best_params['max_depth'] = int(best_params['max_depth'])



BEST PARAMS:  {'bagging_fraction': 0.506555543150077, 'colsample_bytree': 0.717845552080922, 'feature_fraction': 0.6210569597740575, 'gamma': 0.6351261605641609, 'learning_rate': 0.016855486976557935, 'max_depth': 14.0, 'min_child_samples': 160, 'num_leaves': 40, 'reg_alpha': 0.11355857789058366, 'reg_lambda': 0.2014281265579605, 'subsample': 0.9}


In [20]:
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve,scorer
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

def telecom_churn_prediction(algorithm,training_x,testing_x,
                             training_y,testing_y,cols,cf,threshold_plot) :
    
    #model
    algorithm.fit(
    train_X, 
    train_Y, 
    eval_metric="auc", 
    eval_set=[(train_X, train_Y)], 
    verbose=True, 
    early_stopping_rounds = 50)
    
    predictions   = algorithm.predict(testing_x)
    predictions   = predictions.astype(int)
    probabilities = algorithm.predict_proba(testing_x)
    testing_y = testing_y.astype(int)
    #coeffs
    if   cf == "coefficients" :
        coefficients  = pd.DataFrame(algorithm.coef_.ravel())
    elif cf == "features" :
        coefficients  = pd.DataFrame(algorithm.feature_importances_)
        
    column_df     = pd.DataFrame(cols)
    coef_sumry    = (pd.merge(coefficients,column_df,left_index= True,
                              right_index= True, how = "left"))
    coef_sumry.columns = ["coefficients","features"]
    coef_sumry    = coef_sumry.sort_values(by = "coefficients",ascending = False)
    
    print (algorithm)
    print ("\n Classification report : \n",classification_report(testing_y,predictions))
    print ("Accuracy   Score : ",accuracy_score(testing_y,predictions))
    #confusion matrix
    conf_matrix = confusion_matrix(testing_y,predictions)
    #roc_auc_score
    model_roc_auc = roc_auc_score(testing_y,predictions) 
    print ("Area under curve : ",model_roc_auc,"\n")
    fpr,tpr,thresholds = roc_curve(testing_y,probabilities[:,1])
    
    #plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix ,
                        x = ["Not churn","Churn"],
                        y = ["Not churn","Churn"],
                        showscale  = False,colorscale = "Picnic",
                        name = "matrix")
    
    #plot roc curve
    trace2 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2))
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    
    #plot coeffs
    trace4 = go.Bar(x = coef_sumry["features"],y = coef_sumry["coefficients"],
                    name = "coefficients",
                    marker = dict(color = coef_sumry["coefficients"],
                                  colorscale = "Picnic",
                                  line = dict(width = .6,color = "black")))
    
    #subplots
    fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic',
                                            'Feature Importances'))
    
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,1,2)
    fig.append_trace(trace4,2,1)
    
    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 900,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         margin = dict(b = 195))
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis3"].update(dict(showgrid = True,tickfont = dict(size = 10),
                                        tickangle = 90))
    py.iplot(fig)
    
    if threshold_plot == True : 
        visualizer = DiscriminationThreshold(algorithm)
        visualizer.fit(training_x,training_y)
        visualizer.poof()

/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [34]:

best_params = {'bagging_fraction': 0.744118795572853, 'colsample_bytree': 0.7420375024700593, 'feature_fraction': 0.6245510612763201, 'gamma': 0.6909879090881141, 'learning_rate': 0.019619320510941774, 'max_depth': 20.0, 'min_child_samples': 240, 'num_leaves': 90, 'reg_alpha': 0.39880050317381394, 'reg_lambda': 0.38240267515280324, 'subsample': 0.8}

best_params['max_depth'] = int(best_params['max_depth'])

In [35]:
from xgboost import XGBClassifier

clf = XGBClassifier(
    n_estimators=800,
    **best_params
)

clf.fit(X_train, y_train,eval_metric="aucpr", 
    eval_set=[(X_train, y_train)], 
    verbose=True, early_stopping_rounds = 25)





[0]	validation_0-aucpr:0.630843
Will train until validation_0-aucpr hasn't improved in 25 rounds.
[1]	validation_0-aucpr:0.680416
[2]	validation_0-aucpr:0.706329
[3]	validation_0-aucpr:0.720747
[4]	validation_0-aucpr:0.731088
[5]	validation_0-aucpr:0.737733
[6]	validation_0-aucpr:0.74258
[7]	validation_0-aucpr:0.75157
[8]	validation_0-aucpr:0.754189
[9]	validation_0-aucpr:0.757403
[10]	validation_0-aucpr:0.760171
[11]	validation_0-aucpr:0.763636
[12]	validation_0-aucpr:0.766369
[13]	validation_0-aucpr:0.768493
[14]	validation_0-aucpr:0.77145
[15]	validation_0-aucpr:0.77261
[16]	validation_0-aucpr:0.773718
[17]	validation_0-aucpr:0.775313
[18]	validation_0-aucpr:0.776476
[19]	validation_0-aucpr:0.779576
[20]	validation_0-aucpr:0.780481
[21]	validation_0-aucpr:0.782276
[22]	validation_0-aucpr:0.783585
[23]	validation_0-aucpr:0.785164
[24]	validation_0-aucpr:0.786942
[25]	validation_0-aucpr:0.787723
[26]	validation_0-aucpr:0.789304
[27]	validation_0-aucpr:0.791606
[28]	validation_0-aucpr:

[243]	validation_0-aucpr:0.980839
[244]	validation_0-aucpr:0.981148
[245]	validation_0-aucpr:0.981547
[246]	validation_0-aucpr:0.981901
[247]	validation_0-aucpr:0.982259
[248]	validation_0-aucpr:0.982596
[249]	validation_0-aucpr:0.982908
[250]	validation_0-aucpr:0.983196
[251]	validation_0-aucpr:0.983514
[252]	validation_0-aucpr:0.983844
[253]	validation_0-aucpr:0.984102
[254]	validation_0-aucpr:0.984372
[255]	validation_0-aucpr:0.984653
[256]	validation_0-aucpr:0.984917
[257]	validation_0-aucpr:0.985185
[258]	validation_0-aucpr:0.985447
[259]	validation_0-aucpr:0.985742
[260]	validation_0-aucpr:0.985982
[261]	validation_0-aucpr:0.986216
[262]	validation_0-aucpr:0.98641
[263]	validation_0-aucpr:0.986643
[264]	validation_0-aucpr:0.986834
[265]	validation_0-aucpr:0.987055
[266]	validation_0-aucpr:0.987258
[267]	validation_0-aucpr:0.987458
[268]	validation_0-aucpr:0.987687
[269]	validation_0-aucpr:0.987897
[270]	validation_0-aucpr:0.988038
[271]	validation_0-aucpr:0.988211
[272]	validatio

[485]	validation_0-aucpr:0.999203
[486]	validation_0-aucpr:0.999208
[487]	validation_0-aucpr:0.999214
[488]	validation_0-aucpr:0.999223
[489]	validation_0-aucpr:0.999231
[490]	validation_0-aucpr:0.999238
[491]	validation_0-aucpr:0.99924
[492]	validation_0-aucpr:0.999249
[493]	validation_0-aucpr:0.999256
[494]	validation_0-aucpr:0.999261
[495]	validation_0-aucpr:0.99927
[496]	validation_0-aucpr:0.999274
[497]	validation_0-aucpr:0.999282
[498]	validation_0-aucpr:0.999283
[499]	validation_0-aucpr:0.999293
[500]	validation_0-aucpr:0.999299
[501]	validation_0-aucpr:0.999302
[502]	validation_0-aucpr:0.999312
[503]	validation_0-aucpr:0.999318
[504]	validation_0-aucpr:0.999331
[505]	validation_0-aucpr:0.999337
[506]	validation_0-aucpr:0.999349
[507]	validation_0-aucpr:0.999358
[508]	validation_0-aucpr:0.99936
[509]	validation_0-aucpr:0.999365
[510]	validation_0-aucpr:0.999369
[511]	validation_0-aucpr:0.999379
[512]	validation_0-aucpr:0.999382
[513]	validation_0-aucpr:0.999383
[514]	validation_

[727]	validation_0-aucpr:0.999934
[728]	validation_0-aucpr:0.999935
[729]	validation_0-aucpr:0.999936
[730]	validation_0-aucpr:0.999936
[731]	validation_0-aucpr:0.999936
[732]	validation_0-aucpr:0.999937
[733]	validation_0-aucpr:0.999937
[734]	validation_0-aucpr:0.999937
[735]	validation_0-aucpr:0.999938
[736]	validation_0-aucpr:0.999939
[737]	validation_0-aucpr:0.99994
[738]	validation_0-aucpr:0.99994
[739]	validation_0-aucpr:0.999941
[740]	validation_0-aucpr:0.999942
[741]	validation_0-aucpr:0.999942
[742]	validation_0-aucpr:0.999943
[743]	validation_0-aucpr:0.999943
[744]	validation_0-aucpr:0.999944
[745]	validation_0-aucpr:0.999944
[746]	validation_0-aucpr:0.999945
[747]	validation_0-aucpr:0.999945
[748]	validation_0-aucpr:0.999945
[749]	validation_0-aucpr:0.999946
[750]	validation_0-aucpr:0.999946
[751]	validation_0-aucpr:0.999947
[752]	validation_0-aucpr:0.999947
[753]	validation_0-aucpr:0.999947
[754]	validation_0-aucpr:0.999948
[755]	validation_0-aucpr:0.999948
[756]	validation

XGBClassifier(bagging_fraction=0.744118795572853, base_score=0.5,
       booster='gbtree', colsample_bylevel=1, colsample_bynode=1,
       colsample_bytree=0.7420375024700593,
       feature_fraction=0.6245510612763201, gamma=0.6909879090881141,
       learning_rate=0.019619320510941774, max_delta_step=0, max_depth=20,
       min_child_samples=240, min_child_weight=1, missing=None,
       n_estimators=800, n_jobs=1, nthread=None, num_leaves=90,
       objective='binary:logistic', random_state=0,
       reg_alpha=0.39880050317381394, reg_lambda=0.38240267515280324,
       scale_pos_weight=1, seed=None, silent=None, subsample=0.8,
       verbosity=1)

In [36]:
probabilidades   = clf.predict_proba(df_test.drop('isFraud',axis=1))






In [37]:
probabilidades.shape[0]

506691

In [38]:
probabilidades =pd.DataFrame(probabilidades, columns =['no','si'])
df_test = df_test.reset_index().drop(['index'], axis=1)
df_test['isFraud'] = probabilidades.si
resultado = df_test[['TransactionID','isFraud']]
resultado = resultado.set_index('TransactionID')
resultado.to_csv('submission17.csv')


In [24]:
print ("Accuracy   Score : ",accuracy_score(test_Y,prediccion))
print ("Roc AUC   Score : ",roc_auc_score(test_Y,prediccion))

Accuracy   Score :  0.9832849087985425
Roc AUC   Score :  0.8023214688053426


In [250]:
prediccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144899 entries, 0 to 144898
Data columns (total 1 columns):
0    144899 non-null object
dtypes: object(1)
memory usage: 1.1+ MB
